In [7]:
from qick import *
from pickle_parser import *
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/Users/Daniel/qick/.qick_venv/lib/python3.9/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['time']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [8]:
# Pyro to communicate with board
import Pyro4
from qick import QickConfig
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "192.168.2.99"
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)


QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by o

In [9]:
# Import dictionary of sequences from pickle file
import pickle
with open('Sequence_ch1_ch2_mw.pickle', 'rb') as handle:
    imported_seqs = pickle.load(handle)
with open('Sequence_ch1_ch2_mw.pickle', 'rb') as handle:
    raman_cooling_sequences = pickle.load(handle)
# print(imported_seqs)

In [10]:
pulse_seqs = {}
pulse_seqs["DAC_A"] = imported_seqs["MW"]
pulse_seqs["PMOD_0"] = imported_seqs["ch2_Dig"]
pulse_seqs["PMOD_1"] = imported_seqs["ch1_Dig"]

raman_cooling = PickleParse(pulse_seqs)

----- DAC -----
pulse_lens [47.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
times [0.1, 50.65, 54.65, 58.65, 62.65, 66.65, 70.65, 74.65, 78.65, 82.65, 86.65, 90.65]
freqs [50.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0]
finish 94.3
num_pulses 12
ch_type DAC
ch_ref 1
----- PMOD ------
----- PMOD ------
Last finish: 94.3


In [11]:
# Set useful constants
GEN_CH_A = 1
GEN_CH_B = 0
RO_CH_C = 1
RO_CH_D = 0

In [12]:
ch_cfg = raman_cooling.ch_cfg
offset = 38 # Offset DAC by 38 clock cycles for alignment with digital pulses

# Initialise program
prog = QickProgram(soccfg)
prog.declare_gen(ch=GEN_CH_A, nqz=1) # Initialise DAC
phase = prog.deg2reg(0, gen_ch=GEN_CH_A) # TODO: Set here and override later?
gain = 10000 # TODO: Set here and override later?
prog.default_pulse_registers(ch=GEN_CH_A, phase=phase, gain=gain) # Set default pulse parameters
prog.synci(200)  # Give processor some time to configure pulses
prog.regwi(0, 14, 9) # 10 reps, stored in page 0, register 14
prog.label("LOOP_I") # Start of internal loop

# Configure DAC pulses
# TODO: tidy by taking objects out of functions

for channel in ch_cfg:
    ch_ref = ch_cfg[channel]["ch_ref"]
    num_pulses = ch_cfg[channel]["num_pulses"]

    if ch_cfg[channel]["ch_type"] == "DAC":
        for i in range(num_pulses):
            # TODO: Set phase and gain parameters
            # phase = self.deg2reg(self.cfg["res_phase"], gen_ch=GEN_CH_A)
            # gain = self.cfg["pulse_gain"]

            # Pulse parameters
            freq = prog.freq2reg(ch_cfg["DAC_A"]["freqs"][i], gen_ch=ch_ref)
            time = prog.us2cycles(ch_cfg["DAC_A"]["times"][i]) - offset
            pulse_len = prog.us2cycles(ch_cfg["DAC_A"]["pulse_lens"][i], gen_ch=ch_ref)

            # Store pulse parameters in register, then trigger pulse at given time
            prog.set_pulse_registers(ch=ch_ref, freq=freq, style="const", length=pulse_len)
            prog.pulse(ch=ch_ref, t=time)

# Configure digital pulses
seq_0 = DigitalOutput(soccfg, 0, raman_cooling_sequences["ch2_Dig"])
seq_1 = DigitalOutput(soccfg, 1, raman_cooling_sequences["ch1_Dig"])
DigitalOutput.configure(prog, soccfg)

# Synchronise channels and go back to beginning of loop
prog.wait_all()
prog.synci(prog.us2cycles(ch_cfg["DAC_A"]["finish"]))
prog.loopnz(0, 14, "LOOP_I")
prog.end()

# Run program!
prog.config_all(soc)
soc.tproc.start()
print(prog) # Print assembly instructions


// Program

        regwi 1, $23, 0;                        //phase = 0
        regwi 1, $25, 10000;                    //gain = 10000
        synci 200;
        regwi 0, $14, 9;
LOOP_I: regwi 1, $22, 21845333;                 //freq = 21845333
        regwi 1, $26, 618701;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 28877 
        regwi 1, $27, 3;                        //t = 3
        set 1, 1, $22, $23, $0, $25, $26, $27;  //ch = 1, pulse @t = $27
        regwi 1, $22, 21845333;                 //freq = 21845333
        regwi 1, $26, 589885;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 61 
        regwi 1, $27, 20708;                    //t = 20708
        set 1, 1, $22, $23, $0, $25, $26, $27;  //ch = 1, pulse @t = $27
        regwi 1, $22, 24029867;                 //freq = 24029867
        regwi 1, $26, 589885;                   //phrst| stdysel | mode | | outsel = 0b01001 | length = 61 
        regwi 1, $27, 22347;     